In [1]:
import polars as pl
import constants

In [15]:
df = pl.read_parquet(constants.CONTRIBUTIONS_TABLE_PARQUET_PATH)

In [16]:
df

transaction.id,transaction.type,date,amount,committee.id,contributor.name,contributor.address,contributor.city,contributor.state,contributor.zipcode,is.corp,candidate.id,recipient.name,candidate.party,recipient.state,election.type,ico.status
str,str,date,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str
"""comm:1980:100003""","""24K""",1979-06-19,250,100031605,"""ATLANTIC RICHFIELD COMPANY CON…","""612 s flower st, suite 309""","""los angeles""","""CA""",0,"""corp""","""cand337""","""HEFTEL, MR. CECIL LANDAU""","""Democrat""","""HI""","""P""","""I"""
"""comm:1980:100018""","""24K""",1979-08-19,25,100031605,"""ATLANTIC RICHFIELD COMPANY CON…","""612 s flower st, suite 309""","""los angeles""","""CA""",0,"""corp""","""cand131""","""CRANE, PHILIP M.""","""Republican""","""IL""","""P""","""I"""
"""comm:1980:100032""","""24K""",1979-10-19,125,100031605,"""ATLANTIC RICHFIELD COMPANY CON…","""612 s flower st, suite 309""","""los angeles""","""CA""",0,"""corp""","""cand464""","""WYATT, JOE JR.""","""Democrat""","""TX""","""P""","""I"""
"""comm:1980:100047""","""24K""",1979-10-19,100,100031605,"""ATLANTIC RICHFIELD COMPANY CON…","""612 s flower st, suite 309""","""los angeles""","""CA""",0,"""corp""","""cand102""","""MCCLOSKEY, PAUL N JR""","""Republican""","""CA""","""P""","""I"""
"""comm:1980:100061""","""24K""",1980-01-19,100,100031605,"""ATLANTIC RICHFIELD COMPANY CON…","""612 s flower st, suite 309""","""los angeles""","""CA""",0,"""corp""","""cand417""","""FROST, MARTIN""","""Democrat""","""TX""","""P""","""I"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""indv:1980:293350""","""15""",1979-11-29,500,101835283,"""council, james""",null,"""somerset""","""KY""",42501,null,"""cand541""","""ROGERS, HAROLD""","""Republican""","""KY""","""P""","""O"""
"""indv:1980:325906""","""15""",1980-05-16,1000,101653950,"""hamil & mckinney""",null,"""lexington""","""KY""",0,null,"""cand36819""","""BROWN, JOHN YOUNG""","""Democrat""","""KY""","""P""","""C"""
"""indv:1980:44485""","""24T""",1980-07-02,50,101305723,"""mullin for senate cmte""",null,null,null,0,null,"""cand104244""","""MULLIN, JAMES E""","""Republican""","""VT""","""G""","""C"""


In [9]:
agg_donations_df = df.select(['committee.id', 'candidate.id', 'amount']).group_by(['committee.id', 'candidate.id']).sum()

In [12]:
agg_donations_df

committee.id,candidate.id,amount
i64,str,i64
100031930,"""cand39372""",500
100031986,"""cand403""",200
108168941,"""cand251""",500
100031877,"""cand391""",500
40923,"""cand157""",250
…,…,…
100031556,"""cand257""",400
100048276,"""cand515""",200
100050277,"""cand303""",500


In [14]:
76473/(2726 * 1566)

0.017913915382734166

In [10]:
agg_donations_df['committee.id'].unique()

committee.id
i64
935
943
1636
1925
2303
…
35398802066
35398802158
35400803005


In [11]:
agg_donations_df['candidate.id'].unique()

candidate.id
str
"""cand56291"""
"""cand35308"""
"""cand37456"""
"""cand409"""
"""cand37327"""
…
"""cand53339"""
"""cand35929"""
"""cand37272"""


In [122]:
df.group_by(['committee.id', 'candidate.party']).sum().sort(by='amount', descending=True).group_by('committee.id', maintain_order=True).first()[['committee.id', 'candidate.party', 'amount']]

committee.id,candidate.party,amount
i64,str,i64
100031435,"""Republican""",5271071
36035281644,"""Republican""",3182181
100031433,"""Republican""",1488401
2832,"""Democrat""",1458140
100031430,"""Republican""",1129109
…,…,…
100770433,"""Republican""",10
100739327,"""Republican""",10
101339345,"""Democrat""",10


In [107]:
df.null_count()

transaction.id,transaction.type,date,amount,committee.id,contributor.name,contributor.address,contributor.city,contributor.state,contributor.zipcode,is.corp,candidate.id,recipient.name,candidate.party,recipient.state,election.type,ico.status
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,198,55,56,0,35424,0,0,0,0,6,752


In [106]:
df['ico.status'].unique()

ico.status
str
"""I"""
""""""
null
"""O"""
"""C"""
